In [2]:
from pyspark.sql import SparkSession, functions
from pyspark.sql.types import StructField ,StructType, StringType, IntegerType, DateType, DoubleType

In [3]:
spark = SparkSession.builder.master('local[*]').appName('Market Analysis').config('spark.ui.port', '4050').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/03 01:18:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
schema = StructType(
    [StructField('Row ID', IntegerType()),
    StructField('Order ID', StringType()),
    StructField('Order Date', DateType()),
    StructField('Ship Date', DateType()),
    StructField('Ship Mode', StringType()),
    StructField('Customer ID', StringType()),
    StructField('Customer Name', StringType()),
    StructField('Segment', StringType()),
    StructField('Country', StringType()),
    StructField('City', StringType()),
    StructField('State', StringType()),
    StructField('Postal Code', StringType()),
    StructField('Region', StringType()),
    StructField('Product ID', StringType()),
    StructField('Category', StringType()),
    StructField('Sub-Category', StringType()),
    StructField('Product Name', StringType()),
    StructField('GMV', DoubleType()),
    StructField('Units', IntegerType()),
    StructField('Discount', DoubleType()),
    StructField('Profit', DoubleType())]
)

In [5]:
raw_data = spark.read.format('csv')\
        .option('header', True)\
        .option('delimiter', ',')\
        .option('dateFormat', 'yyyy-MM-dd')\
        .schema(schema)\
        .load('./data/raw_data.csv')
                    

In [6]:
raw_data.show(5, False)

+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+-----------------------------------------------------------+--------+-----+--------+--------+
|Row ID|Order ID      |Order Date|Ship Date |Ship Mode     |Customer ID|Customer Name  |Segment  |Country      |City           |State     |Postal Code|Region|Product ID     |Category       |Sub-Category|Product Name                                               |GMV     |Units|Discount|Profit  |
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+-----------------------------------------------------------+--------+-----+--------+--------+
|1     |CA-2016-152156|2016-11-08|2016-11-11|Second Class  |CG-12520   |Claire Gute    |Consumer |United Stat

In [7]:
raw_data.printSchema()

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: date (nullable = true)
 |-- Ship Date: date (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- GMV: double (nullable = true)
 |-- Units: integer (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Profit: double (nullable = true)



In [8]:
raw_data.limit(5).toPandas()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,GMV,Units,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,220.0000
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [9]:
raw_data.createOrReplaceTempView("raw_data")

# Questão 1

In [34]:
df1 = spark.sql('''SELECT COUNT(DISTINCT `Order ID`) AS `Pedidos Unicos`
                    FROM raw_data
                    WHERE YEAR(`Order Date`) BETWEEN 2014 AND 2017''')
df1.show()

+--------------+
|Pedidos Unicos|
+--------------+
|          5009|
+--------------+



# Questão 2

In [46]:
df2 = spark.sql('''SELECT `Order Date`,
                    `Customer Name`,
                    `Product Name`,
                    Discount
                    FROM raw_data
                    WHERE Discount = (SELECT MAX(Discount)
                                        FROM raw_data)
                    
                    ''')
df2.show(df2.count(), False)

+----------+--------------+------------------------------------------+--------+
|Order Date|Customer Name |Product Name                              |Discount|
+----------+--------------+------------------------------------------+--------+
|2017-09-15|Doug O'Connell|"Avery Trapezoid Ring Binder, 3"" Capacity|295.056 |
+----------+--------------+------------------------------------------+--------+



# Questão 3

## A

In [31]:
df3_top_sellers = spark.sql('''
                    WITH top_produtos as (
                    SELECT `Sub-Category`, 
                    COUNT(DISTINCT `Order ID`) as `Pedidos Unicos`
                    FROM raw_data
                    WHERE YEAR(`Order Date`) BETWEEN 2014 AND 2017
                    GROUP BY `Sub-Category`
                    ORDER BY 2 DESC
                    )
                    SELECT * FROM top_produtos
                    LIMIT 5
                    
                    ''')
df3_top_sellers.show()

+------------+--------------+
|Sub-Category|Pedidos Unicos|
+------------+--------------+
|     Binders|          1316|
|       Paper|          1191|
| Furnishings|           877|
|      Phones|           814|
|     Storage|           777|
+------------+--------------+



## B

In [36]:
df3_top_gmv = spark.sql('''
                    WITH top_produtos as (
                    SELECT `Sub-Category`, 
                    SUM(GMV) as `Total GMV`
                    FROM raw_data
                    WHERE YEAR(`Order Date`) BETWEEN 2014 AND 2017
                    GROUP BY `Sub-Category`
                    ORDER BY 2 DESC
                    )
                    SELECT * FROM top_produtos
                    LIMIT 5
                    
                    ''')
df3_top_gmv.show()

+------------+------------------+
|Sub-Category|         Total GMV|
+------------+------------------+
|      Phones| 329753.0880000001|
|      Chairs|328449.10300000076|
|     Storage|216803.21200000012|
|      Tables| 206965.5320000001|
|     Binders|199905.71700000006|
+------------+------------------+



# Questão 4

## A

In [35]:
df4_unique_products = spark.sql('''
                    SELECT COUNT(DISTINCT `Product ID`) AS `Produtos Unicos`
                    FROM raw_data
                    ''')
df4_unique_products.show()

+---------------+
|Produtos Unicos|
+---------------+
|           1862|
+---------------+



## B

In [42]:
df4_top_products = spark.sql('''
                    WITH top_produtos as (
                    SELECT `Product ID`, 
                    SUM(Units) as `Total Units`
                    FROM raw_data
                    WHERE YEAR(`Order Date`) BETWEEN 2014 AND 2017
                    GROUP BY `Product ID`
                    ORDER BY 2 DESC
                    )
                    SELECT * FROM top_produtos
                    LIMIT 10
                    
                    ''')
df4_top_products.show()

+---------------+-----------+
|     Product ID|Total Units|
+---------------+-----------+
|TEC-AC-10003832|         75|
|OFF-PA-10001970|         70|
|OFF-BI-10001524|         67|
|FUR-CH-10002647|         64|
|OFF-BI-10002026|         64|
|FUR-TA-10001095|         61|
|TEC-AC-10002049|         60|
|OFF-BI-10004728|         59|
|FUR-CH-10003774|         59|
|FUR-FU-10001473|         57|
+---------------+-----------+



# Questão 5

## A

In [39]:
df5_unique_customers = spark.sql('''
                    SELECT COUNT(DISTINCT `Customer ID`) AS `Clientes Unicos`
                    FROM raw_data
                    
                    ''')
df5_unique_customers.show()

+---------------+
|Clientes Unicos|
+---------------+
|            793|
+---------------+



## B

In [41]:
df5_top_customers = spark.sql('''
                    WITH top_clientes as (
                    SELECT `Customer ID`, 
                    SUM(GMV) as `Total GMV`
                    FROM raw_data
                    WHERE YEAR(`Order Date`) BETWEEN 2014 AND 2017
                    GROUP BY `Customer ID`
                    ORDER BY 2 DESC
                    )
                    SELECT * FROM top_clientes
                    LIMIT 10
                    
                    ''')
df5_top_customers.show()

+-----------+------------------+
|Customer ID|         Total GMV|
+-----------+------------------+
|   SM-20320|          25043.05|
|   TC-20980|19017.847999999998|
|   RB-19360|         15117.339|
|   TA-21385|          14595.62|
|   AB-10105|14355.610999999997|
|   SC-20095|14142.333999999999|
|   KL-16645|         14071.917|
|   HL-15040|12873.297999999999|
|   SE-20110|12209.438000000002|
|   CC-12370|         12129.072|
+-----------+------------------+

